# 55. エピポーラ幾何の理論
## Epipolar Geometry Theory

---

## 学習目標

このノートブックを完了すると、以下ができるようになります：

- [ ] エピポーラ幾何の基本概念（エピポール、エピポーラ線、エピポーラ面）を理解する
- [ ] 基礎行列（Fundamental Matrix）Fの意味と性質を説明できる
- [ ] 本質行列（Essential Matrix）Eの意味と性質を説明できる
- [ ] エピポーラ拘束条件を数学的に記述できる
- [ ] 8点アルゴリズムでFを推定できる
- [ ] RANSACを用いたロバスト推定を理解する
- [ ] Eの分解からカメラの相対姿勢（R, t）を復元できる

---

## 前提知識

- 51: ピンホールカメラモデルと射影変換
- 53: 3D座標変換と剛体運動（SO(3), SE(3)）
- 54: カメラキャリブレーション（内部パラメータK）
- 線形代数：SVD分解、固有値分解

**難易度**: ★★★★☆（上級）  
**推定学習時間**: 90-120分

---

## 1. エピポーラ幾何とは

**エピポーラ幾何**（Epipolar Geometry）は、2つのカメラ視点から同じ3Dシーンを観測したときに成り立つ幾何学的関係を記述する理論です。

### なぜエピポーラ幾何が重要なのか？

1. **対応点探索の効率化**: 2D画像間の対応点を1次元探索に削減
2. **カメラ姿勢推定**: 2枚の画像から相対的なカメラ位置・姿勢を復元
3. **3D復元の基礎**: ステレオビジョン、SfMの数学的基盤
4. **外れ値検出**: 誤対応の検出と除去

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import Tuple, List, Optional
import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision=4, suppress=True)

print("ライブラリのインポート完了")

---

## 2. エピポーラ幾何の基本要素

### 2.1 セットアップ

2台のカメラが同じ3D点 $\mathbf{X}$ を観測する状況を考えます：

- **カメラ1**: 中心 $\mathbf{C}_1$、画像点 $\mathbf{x}_1$
- **カメラ2**: 中心 $\mathbf{C}_2$、画像点 $\mathbf{x}_2$

### 2.2 基本要素の定義

| 要素 | 定義 | 性質 |
|------|------|------|
| **ベースライン** | $\mathbf{C}_1$と$\mathbf{C}_2$を結ぶ線分 | 2つのカメラ中心を接続 |
| **エピポール** $\mathbf{e}_1, \mathbf{e}_2$ | ベースラインと各画像面の交点 | 相手カメラの投影位置 |
| **エピポーラ面** | $\mathbf{C}_1, \mathbf{C}_2, \mathbf{X}$を含む平面 | 各3D点に対して一意に決まる |
| **エピポーラ線** $\mathbf{l}_1, \mathbf{l}_2$ | エピポーラ面と各画像面の交線 | 対応点はこの線上に存在 |

In [ ]:
def visualize_epipolar_geometry():
    """エピポーラ幾何の3D可視化"""
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # カメラ中心
    C1 = np.array([0, 0, 0])
    C2 = np.array([2, 0, 0.3])
    
    # 3D点
    X = np.array([1.0, 3.0, 1.5])
    
    # 画像面（簡略化：z=1の平面）
    # カメラ1の画像面
    img1_corners = np.array([
        [-0.5, 1, -0.5],
        [0.5, 1, -0.5],
        [0.5, 1, 0.5],
        [-0.5, 1, 0.5]
    ])
    
    # カメラ2の画像面
    img2_corners = np.array([
        [1.5, 1, -0.5 + 0.3],
        [2.5, 1, -0.5 + 0.3],
        [2.5, 1, 0.5 + 0.3],
        [1.5, 1, 0.5 + 0.3]
    ])
    
    # 投影点（簡略化）
    # カメラ1への投影
    t1 = 1.0 / X[1]  # y=1の平面までのスケール
    x1 = C1 + t1 * (X - C1)
    
    # カメラ2への投影
    t2 = 1.0 / (X[1] - C2[1]) if X[1] != C2[1] else 0
    x2 = C2 + t2 * (X - C2)
    x2[1] = 1  # y=1の平面に固定
    
    # エピポール
    # e1: C2をカメラ1に投影
    t_e1 = 1.0 / C2[1] if C2[1] != 0 else 0
    e1 = C1 + t_e1 * (C2 - C1)
    e1[1] = 1
    
    # e2: C1をカメラ2に投影  
    t_e2 = 1.0 / (-C2[1]) if C2[1] != 0 else 0
    e2 = C2 + t_e2 * (C1 - C2)
    e2[1] = 1
    
    # プロット
    # カメラ中心
    ax.scatter(*C1, color='blue', s=200, marker='o', label='Camera 1 Center')
    ax.scatter(*C2, color='green', s=200, marker='o', label='Camera 2 Center')
    
    # 3D点
    ax.scatter(*X, color='red', s=200, marker='*', label='3D Point X')
    
    # ベースライン
    ax.plot([C1[0], C2[0]], [C1[1], C2[1]], [C1[2], C2[2]], 
            'k-', linewidth=3, label='Baseline')
    
    # 投影光線
    ax.plot([C1[0], X[0]], [C1[1], X[1]], [C1[2], X[2]], 
            'b--', linewidth=2, alpha=0.7, label='Ray from C1')
    ax.plot([C2[0], X[0]], [C2[1], X[1]], [C2[2], X[2]], 
            'g--', linewidth=2, alpha=0.7, label='Ray from C2')
    
    # 投影点
    ax.scatter(*x1, color='blue', s=100, marker='x')
    ax.scatter(*x2, color='green', s=100, marker='x')
    
    # エピポーラ面（三角形として描画）
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    verts = [[C1, C2, X]]
    poly = Poly3DCollection(verts, alpha=0.2, facecolor='yellow', edgecolor='orange')
    ax.add_collection3d(poly)
    
    # 画像面（半透明）
    for corners, color in [(img1_corners, 'blue'), (img2_corners, 'green')]:
        verts = [corners]
        poly = Poly3DCollection(verts, alpha=0.1, facecolor=color, edgecolor=color)
        ax.add_collection3d(poly)
    
    # ラベル
    ax.text(*C1, '  C₁', fontsize=12)
    ax.text(*C2, '  C₂', fontsize=12)
    ax.text(*X, '  X', fontsize=12)
    ax.text(*x1, '  x₁', fontsize=10)
    ax.text(*x2, '  x₂', fontsize=10)
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Epipolar Geometry Visualization\n(Yellow: Epipolar Plane)', fontsize=14)
    ax.legend(loc='upper left')
    
    # 視点調整
    ax.view_init(elev=20, azim=45)
    
    plt.tight_layout()
    plt.show()

visualize_epipolar_geometry()

### 2.3 エピポーラ拘束の幾何学的意味

**核心的な洞察**：

画像1上の点 $\mathbf{x}_1$ が与えられたとき、対応する3D点 $\mathbf{X}$ は光線 $\mathbf{C}_1 \mathbf{x}_1$ 上のどこかに存在します。

この光線をカメラ2に投影すると、**エピポーラ線** $\mathbf{l}_2$ になります。

したがって、$\mathbf{x}_2$ は必ず $\mathbf{l}_2$ 上に存在します。

$$\mathbf{x}_2 \in \mathbf{l}_2$$

これが**エピポーラ拘束**の本質です。

In [ ]:
def visualize_epipolar_constraint_2d():
    """エピポーラ拘束の2D可視化（画像ペア）"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # 画像サイズ
    img_size = (640, 480)
    
    # 画像1の点
    x1 = np.array([320, 200])  # 中央上部
    
    # エピポール（例）
    e1 = np.array([700, 240])  # 画像外（右側）
    e2 = np.array([-60, 240])  # 画像外（左側）
    
    # 画像1
    ax1.set_xlim(0, img_size[0])
    ax1.set_ylim(img_size[1], 0)  # 画像座標系
    ax1.set_aspect('equal')
    ax1.add_patch(plt.Rectangle((0, 0), img_size[0], img_size[1], 
                                 fill=True, facecolor='lightgray', edgecolor='black'))
    
    # 点とエピポーラ線
    ax1.scatter(*x1, color='red', s=200, zorder=5, label='Point x₁')
    ax1.scatter(*e1, color='blue', s=100, marker='x', zorder=5, label='Epipole e₁')
    
    # x1を通るエピポーラ線（e1方向へ）
    direction = e1 - x1
    t_range = np.linspace(-2, 2, 100)
    line_points = x1[:, np.newaxis] + direction[:, np.newaxis] * t_range
    ax1.plot(line_points[0], line_points[1], 'b-', linewidth=2, alpha=0.5)
    
    ax1.set_title('Image 1', fontsize=14)
    ax1.set_xlabel('u (pixels)')
    ax1.set_ylabel('v (pixels)')
    ax1.legend()
    
    # 画像2
    ax2.set_xlim(0, img_size[0])
    ax2.set_ylim(img_size[1], 0)
    ax2.set_aspect('equal')
    ax2.add_patch(plt.Rectangle((0, 0), img_size[0], img_size[1], 
                                 fill=True, facecolor='lightgray', edgecolor='black'))
    
    # エピポーラ線（x1に対応）
    # 簡略化：ほぼ水平なエピポーラ線
    x2_candidates = np.array([[200, 180], [350, 185], [500, 190]])  # 線上の候補点
    x2_true = np.array([350, 185])  # 真の対応点
    
    ax2.scatter(*e2, color='blue', s=100, marker='x', zorder=5, label='Epipole e₂')
    
    # エピポーラ線
    line_y = 185
    ax2.axhline(y=line_y, color='green', linewidth=3, alpha=0.5, label='Epipolar line l₂')
    
    # 対応点候補
    ax2.scatter(x2_candidates[:, 0], x2_candidates[:, 1], 
                color='orange', s=100, alpha=0.5, label='Candidate points')
    ax2.scatter(*x2_true, color='red', s=200, zorder=5, label='True correspondence x₂')
    
    # 探索範囲の注釈
    ax2.annotate('', xy=(600, line_y), xytext=(50, line_y),
                 arrowprops=dict(arrowstyle='<->', color='green', lw=2))
    ax2.text(320, line_y - 30, '1D Search on Epipolar Line', 
             ha='center', fontsize=11, color='green')
    
    ax2.set_title('Image 2', fontsize=14)
    ax2.set_xlabel('u (pixels)')
    ax2.set_ylabel('v (pixels)')
    ax2.legend()
    
    plt.suptitle('Epipolar Constraint: Search for Correspondence', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("エピポーラ拘束により、2D探索が1D探索に削減されます！")
    print(f"探索空間: {img_size[0]} × {img_size[1]} = {img_size[0] * img_size[1]:,} pixels")
    print(f"→ エピポーラ線上: ~{img_size[0]} pixels（{img_size[0] * img_size[1] / img_size[0]:.0f}倍の効率化）")

visualize_epipolar_constraint_2d()

---

## 3. 基礎行列（Fundamental Matrix）F

### 3.1 定義

**基礎行列** $\mathbf{F}$ は、2つの画像間のエピポーラ幾何を完全に記述する $3 \times 3$ 行列です。

#### エピポーラ拘束（代数形式）

$$\mathbf{x}_2^\top \mathbf{F} \mathbf{x}_1 = 0$$

ここで：
- $\mathbf{x}_1 = (u_1, v_1, 1)^\top$ : 画像1の同次座標
- $\mathbf{x}_2 = (u_2, v_2, 1)^\top$ : 画像2の同次座標

### 3.2 エピポーラ線の計算

- **画像2のエピポーラ線**: $\mathbf{l}_2 = \mathbf{F} \mathbf{x}_1$
- **画像1のエピポーラ線**: $\mathbf{l}_1 = \mathbf{F}^\top \mathbf{x}_2$

直線 $\mathbf{l} = (a, b, c)^\top$ は $ax + by + c = 0$ を表します。

### 3.3 基礎行列の性質

| 性質 | 説明 |
|------|------|
| **ランク** | $\text{rank}(\mathbf{F}) = 2$（特異行列） |
| **自由度** | 7 DOF（スケール不定性で-1、ランク2拘束で-1） |
| **エピポール** | $\mathbf{F} \mathbf{e}_1 = \mathbf{0}$, $\mathbf{F}^\top \mathbf{e}_2 = \mathbf{0}$ |

In [ ]:
def skew_symmetric(v: np.ndarray) -> np.ndarray:
    """3Dベクトルから反対称行列を生成"""
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def compute_fundamental_matrix(K1: np.ndarray, K2: np.ndarray, 
                                R: np.ndarray, t: np.ndarray) -> np.ndarray:
    """カメラパラメータから基礎行列を計算
    
    F = K2^(-T) [t]_x R K1^(-1)
    """
    t_skew = skew_symmetric(t)
    E = t_skew @ R  # 本質行列
    F = np.linalg.inv(K2).T @ E @ np.linalg.inv(K1)
    return F / F[2, 2]  # 正規化

def compute_epipolar_line(F: np.ndarray, x: np.ndarray, image: int = 2) -> np.ndarray:
    """エピポーラ線を計算
    
    Args:
        F: 基礎行列
        x: 点の同次座標 (3,)
        image: 1なら画像1のエピポーラ線、2なら画像2のエピポーラ線
    
    Returns:
        l: エピポーラ線 (a, b, c) where ax + by + c = 0
    """
    if image == 2:
        return F @ x
    else:
        return F.T @ x

# 例: カメラパラメータの設定
K = np.array([
    [800, 0, 320],
    [0, 800, 240],
    [0, 0, 1]
])

# カメラ2の相対姿勢（カメラ1からの変換）
theta = np.radians(10)  # 10度回転
R = np.array([
    [np.cos(theta), 0, np.sin(theta)],
    [0, 1, 0],
    [-np.sin(theta), 0, np.cos(theta)]
])  # Y軸周りの回転

t = np.array([0.5, 0, 0.1])  # 右へ0.5、前へ0.1移動

# 基礎行列の計算
F = compute_fundamental_matrix(K, K, R, t)

print("カメラ内部パラメータ K:")
print(K)
print(f"\n回転行列 R (Y軸周り{np.degrees(theta):.1f}度):")
print(R)
print(f"\n並進ベクトル t: {t}")
print("\n基礎行列 F:")
print(F)
print(f"\nrank(F) = {np.linalg.matrix_rank(F)}")

In [ ]:
def verify_epipolar_constraint(F: np.ndarray, x1: np.ndarray, x2: np.ndarray):
    """エピポーラ拘束の検証"""
    # 同次座標に変換
    x1_h = np.append(x1, 1) if len(x1) == 2 else x1
    x2_h = np.append(x2, 1) if len(x2) == 2 else x2
    
    # x2^T F x1
    constraint = x2_h @ F @ x1_h
    return constraint

def visualize_epipolar_lines():
    """複数の点に対するエピポーラ線の可視化"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    img_size = (640, 480)
    
    # テスト点（画像1）
    test_points = np.array([
        [200, 150],
        [320, 240],
        [450, 350],
        [100, 400]
    ])
    
    colors = ['red', 'green', 'blue', 'purple']
    
    # 画像1: 点をプロット
    ax1.set_xlim(0, img_size[0])
    ax1.set_ylim(img_size[1], 0)
    ax1.set_aspect('equal')
    ax1.add_patch(plt.Rectangle((0, 0), img_size[0], img_size[1], 
                                 fill=True, facecolor='lightgray', edgecolor='black'))
    
    for i, (pt, color) in enumerate(zip(test_points, colors)):
        ax1.scatter(*pt, color=color, s=150, zorder=5, label=f'Point {i+1}')
    
    # エピポールを計算（Fの右零空間）
    U, S, Vt = np.linalg.svd(F)
    e1 = Vt[-1]  # 右零空間（F e1 = 0）
    e1 = e1 / e1[2]  # 正規化
    
    # エピポールの可視化（画像範囲外でも方向を示す）
    if 0 <= e1[0] <= img_size[0] and 0 <= e1[1] <= img_size[1]:
        ax1.scatter(e1[0], e1[1], color='black', s=200, marker='x', zorder=10, label='Epipole e₁')
    else:
        ax1.annotate(f'e₁ → ({e1[0]:.0f}, {e1[1]:.0f})', 
                     xy=(img_size[0]-10, img_size[1]//2), fontsize=10,
                     bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
    
    ax1.set_title('Image 1: Points', fontsize=14)
    ax1.set_xlabel('u (pixels)')
    ax1.set_ylabel('v (pixels)')
    ax1.legend(loc='upper left')
    
    # 画像2: エピポーラ線をプロット
    ax2.set_xlim(0, img_size[0])
    ax2.set_ylim(img_size[1], 0)
    ax2.set_aspect('equal')
    ax2.add_patch(plt.Rectangle((0, 0), img_size[0], img_size[1], 
                                 fill=True, facecolor='lightgray', edgecolor='black'))
    
    for i, (pt, color) in enumerate(zip(test_points, colors)):
        # 同次座標
        x1_h = np.array([pt[0], pt[1], 1])
        
        # エピポーラ線
        l2 = compute_epipolar_line(F, x1_h, image=2)
        a, b, c = l2
        
        # 直線を描画（ax + by + c = 0 → y = -(ax + c) / b）
        if abs(b) > 1e-6:
            x_vals = np.array([0, img_size[0]])
            y_vals = -(a * x_vals + c) / b
        else:
            # 垂直線
            x_vals = np.array([-c/a, -c/a])
            y_vals = np.array([0, img_size[1]])
        
        ax2.plot(x_vals, y_vals, color=color, linewidth=2, 
                 label=f'Epipolar line for Point {i+1}')
    
    # エピポールe2を計算（F^Tの右零空間）
    U, S, Vt = np.linalg.svd(F.T)
    e2 = Vt[-1]
    e2 = e2 / e2[2]
    
    if 0 <= e2[0] <= img_size[0] and 0 <= e2[1] <= img_size[1]:
        ax2.scatter(e2[0], e2[1], color='black', s=200, marker='x', zorder=10, label='Epipole e₂')
    else:
        ax2.annotate(f'e₂ → ({e2[0]:.0f}, {e2[1]:.0f})', 
                     xy=(10, img_size[1]//2), fontsize=10,
                     bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
    
    ax2.set_title('Image 2: Epipolar Lines', fontsize=14)
    ax2.set_xlabel('u (pixels)')
    ax2.set_ylabel('v (pixels)')
    ax2.legend(loc='upper right')
    
    plt.suptitle('Fundamental Matrix: Point → Epipolar Line Mapping', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("\n全てのエピポーラ線はエピポールe₂を通過します（画像外の場合あり）")

visualize_epipolar_lines()

---

## 4. 本質行列（Essential Matrix）E

### 4.1 定義

**本質行列** $\mathbf{E}$ は、**正規化画像座標**（カメラ内部パラメータで正規化済み）に対するエピポーラ拘束を記述します。

#### 正規化座標

$$\hat{\mathbf{x}} = \mathbf{K}^{-1} \mathbf{x}$$

#### エピポーラ拘束（本質行列）

$$\hat{\mathbf{x}}_2^\top \mathbf{E} \hat{\mathbf{x}}_1 = 0$$

### 4.2 本質行列の構造

$$\mathbf{E} = [\mathbf{t}]_\times \mathbf{R} = \mathbf{R} [\mathbf{R}^\top \mathbf{t}]_\times$$

ここで：
- $\mathbf{R}$: カメラ2のカメラ1に対する回転
- $\mathbf{t}$: カメラ2のカメラ1に対する並進
- $[\cdot]_\times$: 反対称行列（外積を行列積で表現）

### 4.3 基礎行列との関係

$$\mathbf{F} = \mathbf{K}_2^{-\top} \mathbf{E} \mathbf{K}_1^{-1}$$

または

$$\mathbf{E} = \mathbf{K}_2^\top \mathbf{F} \mathbf{K}_1$$

### 4.4 本質行列の性質

| 性質 | 説明 |
|------|------|
| **ランク** | $\text{rank}(\mathbf{E}) = 2$ |
| **特異値** | 2つの等しい非零特異値：$\sigma_1 = \sigma_2$, $\sigma_3 = 0$ |
| **自由度** | 5 DOF（回転3 + 並進2、スケール不定） |

In [ ]:
def compute_essential_matrix(R: np.ndarray, t: np.ndarray) -> np.ndarray:
    """本質行列を計算: E = [t]_x R"""
    t_skew = skew_symmetric(t)
    E = t_skew @ R
    return E

def verify_essential_matrix_properties(E: np.ndarray):
    """本質行列の性質を検証"""
    U, S, Vt = np.linalg.svd(E)
    
    print("=== 本質行列の性質検証 ===")
    print(f"\n特異値: {S}")
    print(f"σ₁ ≈ σ₂: {np.isclose(S[0], S[1], rtol=0.1)}")
    print(f"σ₃ ≈ 0: {np.isclose(S[2], 0, atol=1e-10)}")
    print(f"\nrank(E) = {np.sum(S > 1e-10)}")
    
    # det(E) = 0 の検証
    print(f"\ndet(E) = {np.linalg.det(E):.6f} (should be ≈ 0)")
    
    # 2E E^T E - trace(E E^T) E = 0 の検証（内部拘束）
    EET = E @ E.T
    constraint = 2 * E @ E.T @ E - np.trace(EET) * E
    print(f"\n内部拘束 ||2EE^TE - tr(EE^T)E|| = {np.linalg.norm(constraint):.6e} (should be ≈ 0)")

# 本質行列の計算と検証
E = compute_essential_matrix(R, t)

print("本質行列 E:")
print(E)
print()

verify_essential_matrix_properties(E)

# F と E の関係を検証
print("\n=== F と E の関係検証 ===")
E_from_F = K.T @ F @ K
E_from_F = E_from_F / np.linalg.norm(E_from_F) * np.linalg.norm(E)  # スケール調整
print(f"E (直接計算):")
print(E / np.linalg.norm(E))
print(f"\nK^T F K (F から計算):")
print(E_from_F / np.linalg.norm(E_from_F))

---

## 5. 8点アルゴリズム

### 5.1 概要

**8点アルゴリズム**は、8組以上の対応点から基礎行列 $\mathbf{F}$ を推定する最も基本的な手法です。

### 5.2 アルゴリズム

#### ステップ1: エピポーラ拘束の線形化

$$\mathbf{x}_2^\top \mathbf{F} \mathbf{x}_1 = 0$$

を展開すると：

$$u_2 u_1 f_{11} + u_2 v_1 f_{12} + u_2 f_{13} + v_2 u_1 f_{21} + v_2 v_1 f_{22} + v_2 f_{23} + u_1 f_{31} + v_1 f_{32} + f_{33} = 0$$

#### ステップ2: 行列形式

$$\mathbf{A} \mathbf{f} = \mathbf{0}$$

ここで $\mathbf{f} = (f_{11}, f_{12}, ..., f_{33})^\top$ は $\mathbf{F}$ の要素をベクトル化したものです。

#### ステップ3: SVD による解

$\mathbf{A} = \mathbf{U} \mathbf{\Sigma} \mathbf{V}^\top$ のSVD分解で、$\mathbf{V}$ の最後の列が $\mathbf{f}$ の解。

#### ステップ4: ランク2への射影

$\mathbf{F}$ の SVD で最小特異値を0に設定し、ランク2を保証。

In [ ]:
def normalize_points(points: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """点の正規化（数値安定性のため）
    
    平均を原点に、平均距離を√2にスケール
    
    Returns:
        normalized_points: 正規化された点
        T: 正規化変換行列
    """
    centroid = np.mean(points, axis=0)
    centered = points - centroid
    mean_dist = np.mean(np.linalg.norm(centered, axis=1))
    scale = np.sqrt(2) / mean_dist if mean_dist > 0 else 1.0
    
    T = np.array([
        [scale, 0, -scale * centroid[0]],
        [0, scale, -scale * centroid[1]],
        [0, 0, 1]
    ])
    
    # 同次座標で変換
    points_h = np.hstack([points, np.ones((len(points), 1))])
    normalized = (T @ points_h.T).T
    
    return normalized[:, :2], T

def eight_point_algorithm(pts1: np.ndarray, pts2: np.ndarray, 
                          normalize: bool = True) -> np.ndarray:
    """8点アルゴリズムによる基礎行列の推定
    
    Args:
        pts1: 画像1の対応点 (N, 2)
        pts2: 画像2の対応点 (N, 2)
        normalize: 正規化を行うか（推奨: True）
    
    Returns:
        F: 基礎行列 (3, 3)
    """
    assert len(pts1) >= 8, "8点以上必要です"
    assert len(pts1) == len(pts2), "対応点の数が一致しません"
    
    # 1. 正規化
    if normalize:
        pts1_norm, T1 = normalize_points(pts1)
        pts2_norm, T2 = normalize_points(pts2)
    else:
        pts1_norm = pts1
        pts2_norm = pts2
        T1 = T2 = np.eye(3)
    
    # 2. 行列Aの構築
    n = len(pts1)
    A = np.zeros((n, 9))
    
    for i in range(n):
        u1, v1 = pts1_norm[i]
        u2, v2 = pts2_norm[i]
        A[i] = [u2*u1, u2*v1, u2, v2*u1, v2*v1, v2, u1, v1, 1]
    
    # 3. SVD で解を求める
    U, S, Vt = np.linalg.svd(A)
    F = Vt[-1].reshape(3, 3)
    
    # 4. ランク2への射影（特異値拘束）
    U_f, S_f, Vt_f = np.linalg.svd(F)
    S_f[2] = 0  # 最小特異値を0に
    F = U_f @ np.diag(S_f) @ Vt_f
    
    # 5. 正規化の逆変換
    if normalize:
        F = T2.T @ F @ T1
    
    # 正規化
    F = F / F[2, 2] if abs(F[2, 2]) > 1e-10 else F / np.linalg.norm(F)
    
    return F

print("8点アルゴリズムの実装完了")

In [ ]:
def generate_synthetic_correspondences(K: np.ndarray, R: np.ndarray, t: np.ndarray,
                                       n_points: int = 50, 
                                       noise_std: float = 0.0) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """合成対応点の生成
    
    Returns:
        pts1: 画像1の点
        pts2: 画像2の点
        X_3d: 3D点
    """
    # ランダムな3D点（カメラ前方）
    np.random.seed(42)
    X_3d = np.random.randn(n_points, 3)
    X_3d[:, 2] = np.abs(X_3d[:, 2]) + 3  # Z > 3 (カメラ前方)
    X_3d[:, 0] *= 2  # Xの範囲を広げる
    X_3d[:, 1] *= 1.5  # Yの範囲を広げる
    
    # カメラ1への投影（世界座標 = カメラ1座標と仮定）
    pts1_h = (K @ X_3d.T).T
    pts1 = pts1_h[:, :2] / pts1_h[:, 2:3]
    
    # カメラ2への投影
    # X_cam2 = R @ X_cam1 + t
    X_cam2 = (R @ X_3d.T).T + t
    pts2_h = (K @ X_cam2.T).T
    pts2 = pts2_h[:, :2] / pts2_h[:, 2:3]
    
    # ノイズ追加
    if noise_std > 0:
        pts1 += np.random.randn(*pts1.shape) * noise_std
        pts2 += np.random.randn(*pts2.shape) * noise_std
    
    # 画像範囲内の点のみフィルタリング
    img_w, img_h = 640, 480
    valid = ((pts1[:, 0] >= 0) & (pts1[:, 0] < img_w) & 
             (pts1[:, 1] >= 0) & (pts1[:, 1] < img_h) &
             (pts2[:, 0] >= 0) & (pts2[:, 0] < img_w) & 
             (pts2[:, 1] >= 0) & (pts2[:, 1] < img_h))
    
    return pts1[valid], pts2[valid], X_3d[valid]

# 合成データの生成
pts1, pts2, X_3d = generate_synthetic_correspondences(K, R, t, n_points=100, noise_std=0.5)

print(f"生成された対応点: {len(pts1)} 組")

# 8点アルゴリズムの実行
F_estimated = eight_point_algorithm(pts1, pts2, normalize=True)

print("\n推定された基礎行列 F:")
print(F_estimated)

# 真の基礎行列との比較
F_true = compute_fundamental_matrix(K, K, R, t)
F_true = F_true / np.linalg.norm(F_true)
F_estimated_norm = F_estimated / np.linalg.norm(F_estimated)

print("\n真の基礎行列 F (正規化):")
print(F_true)

# 符号の不定性を考慮した比較
error1 = np.linalg.norm(F_estimated_norm - F_true)
error2 = np.linalg.norm(F_estimated_norm + F_true)
error = min(error1, error2)
print(f"\n推定誤差 (Frobenius norm): {error:.6f}")

In [ ]:
def visualize_correspondences_and_epipolar_lines(pts1, pts2, F, n_show=8):
    """対応点とエピポーラ線の可視化"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    img_size = (640, 480)
    
    # ランダムに点を選択
    indices = np.random.choice(len(pts1), min(n_show, len(pts1)), replace=False)
    colors = plt.cm.tab10(np.linspace(0, 1, len(indices)))
    
    # 画像1
    ax1.set_xlim(0, img_size[0])
    ax1.set_ylim(img_size[1], 0)
    ax1.set_aspect('equal')
    ax1.add_patch(plt.Rectangle((0, 0), img_size[0], img_size[1], 
                                 fill=True, facecolor='#f0f0f0', edgecolor='black'))
    
    # 画像2
    ax2.set_xlim(0, img_size[0])
    ax2.set_ylim(img_size[1], 0)
    ax2.set_aspect('equal')
    ax2.add_patch(plt.Rectangle((0, 0), img_size[0], img_size[1], 
                                 fill=True, facecolor='#f0f0f0', edgecolor='black'))
    
    for i, (idx, color) in enumerate(zip(indices, colors)):
        p1, p2 = pts1[idx], pts2[idx]
        
        # 点をプロット
        ax1.scatter(*p1, color=color, s=100, zorder=5)
        ax2.scatter(*p2, color=color, s=100, zorder=5, marker='x', linewidths=2)
        
        # 画像2のエピポーラ線
        x1_h = np.array([p1[0], p1[1], 1])
        l2 = F @ x1_h
        a, b, c = l2
        
        if abs(b) > 1e-6:
            x_vals = np.array([0, img_size[0]])
            y_vals = -(a * x_vals + c) / b
        else:
            x_vals = np.array([-c/a, -c/a])
            y_vals = np.array([0, img_size[1]])
        
        ax2.plot(x_vals, y_vals, color=color, linewidth=1.5, alpha=0.7)
        
        # エピポーラ拘束の検証
        x2_h = np.array([p2[0], p2[1], 1])
        constraint = x2_h @ F @ x1_h
        
        # 点からエピポーラ線までの距離
        dist = abs(a * p2[0] + b * p2[1] + c) / np.sqrt(a**2 + b**2)
        
        if i == 0:
            print(f"点{idx}: 拘束値={constraint:.4f}, エピポーラ線への距離={dist:.2f}px")
    
    ax1.set_title('Image 1: Points (circles)', fontsize=14)
    ax1.set_xlabel('u (pixels)')
    ax1.set_ylabel('v (pixels)')
    
    ax2.set_title('Image 2: Points (×) and Epipolar Lines', fontsize=14)
    ax2.set_xlabel('u (pixels)')
    ax2.set_ylabel('v (pixels)')
    
    plt.suptitle('8-Point Algorithm: Estimated Epipolar Lines', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    # 全点の平均距離
    total_dist = 0
    for i in range(len(pts1)):
        x1_h = np.array([pts1[i, 0], pts1[i, 1], 1])
        x2_h = np.array([pts2[i, 0], pts2[i, 1], 1])
        l2 = F @ x1_h
        a, b, c = l2
        dist = abs(a * pts2[i, 0] + b * pts2[i, 1] + c) / np.sqrt(a**2 + b**2)
        total_dist += dist
    
    print(f"\n平均エピポーラ距離: {total_dist / len(pts1):.4f} pixels")

visualize_correspondences_and_epipolar_lines(pts1, pts2, F_estimated)

---

## 6. RANSACによるロバスト推定

### 6.1 なぜRANSACが必要か

実際のデータには**外れ値（outliers）**が含まれます：
- 誤対応（特徴点マッチングのエラー）
- 動く物体
- 反射や遮蔽

8点アルゴリズムは最小二乗法ベースのため、外れ値に弱いです。

### 6.2 RANSACアルゴリズム

```
for i = 1 to max_iterations:
    1. ランダムに8点を選択
    2. 8点アルゴリズムでFを計算
    3. 全点に対してエピポーラ距離を計算
    4. 閾値以下の点をインライアとしてカウント
    5. インライア数が最大なら、このFを保持

最終的に、全インライアでFを再計算
```

### 6.3 必要な反復回数

$$k = \frac{\log(1 - p)}{\log(1 - w^n)}$$

ここで：
- $p$: 成功確率（通常0.99）
- $w$: インライア率
- $n$: サンプル数（8点アルゴリズムでは8）

In [ ]:
def compute_epipolar_distances(F: np.ndarray, pts1: np.ndarray, pts2: np.ndarray) -> np.ndarray:
    """各対応点のエピポーラ距離を計算
    
    Sampson距離（1次近似）を使用
    """
    n = len(pts1)
    distances = np.zeros(n)
    
    for i in range(n):
        x1_h = np.array([pts1[i, 0], pts1[i, 1], 1])
        x2_h = np.array([pts2[i, 0], pts2[i, 1], 1])
        
        # Sampson距離
        Fx1 = F @ x1_h
        Ftx2 = F.T @ x2_h
        
        numerator = (x2_h @ F @ x1_h) ** 2
        denominator = Fx1[0]**2 + Fx1[1]**2 + Ftx2[0]**2 + Ftx2[1]**2
        
        distances[i] = numerator / denominator if denominator > 1e-10 else 0
    
    return np.sqrt(distances)

def ransac_fundamental_matrix(pts1: np.ndarray, pts2: np.ndarray,
                               threshold: float = 3.0,
                               max_iterations: int = 1000,
                               confidence: float = 0.99) -> Tuple[np.ndarray, np.ndarray]:
    """RANSACによる基礎行列のロバスト推定
    
    Returns:
        F: 基礎行列
        inlier_mask: インライアのマスク
    """
    n_points = len(pts1)
    best_F = None
    best_inliers = np.zeros(n_points, dtype=bool)
    best_n_inliers = 0
    
    for iteration in range(max_iterations):
        # 1. ランダムに8点選択
        indices = np.random.choice(n_points, 8, replace=False)
        
        # 2. 8点アルゴリズム
        try:
            F = eight_point_algorithm(pts1[indices], pts2[indices], normalize=True)
        except:
            continue
        
        # 3. エピポーラ距離の計算
        distances = compute_epipolar_distances(F, pts1, pts2)
        
        # 4. インライアの判定
        inliers = distances < threshold
        n_inliers = np.sum(inliers)
        
        # 5. 最良モデルの更新
        if n_inliers > best_n_inliers:
            best_n_inliers = n_inliers
            best_F = F
            best_inliers = inliers
            
            # 早期終了の判定
            inlier_ratio = n_inliers / n_points
            if inlier_ratio > 0.9:  # 90%以上がインライア
                break
    
    # 6. 全インライアでFを再計算
    if np.sum(best_inliers) >= 8:
        best_F = eight_point_algorithm(pts1[best_inliers], pts2[best_inliers], normalize=True)
    
    return best_F, best_inliers

print("RANSAC実装完了")

In [ ]:
def add_outliers(pts1: np.ndarray, pts2: np.ndarray, 
                 outlier_ratio: float = 0.3) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """外れ値を追加"""
    n = len(pts1)
    n_outliers = int(n * outlier_ratio)
    
    # 外れ値のインデックス
    outlier_indices = np.random.choice(n, n_outliers, replace=False)
    
    # コピー
    pts1_out = pts1.copy()
    pts2_out = pts2.copy()
    
    # 外れ値：ランダムな位置に変更
    pts2_out[outlier_indices] = np.random.rand(n_outliers, 2) * [640, 480]
    
    # インライアマスク（真の値）
    inlier_mask_true = np.ones(n, dtype=bool)
    inlier_mask_true[outlier_indices] = False
    
    return pts1_out, pts2_out, inlier_mask_true

# 外れ値を追加したデータ
pts1_noisy, pts2_noisy, true_inliers = add_outliers(pts1, pts2, outlier_ratio=0.3)

print(f"全点数: {len(pts1_noisy)}")
print(f"真のインライア数: {np.sum(true_inliers)}")
print(f"外れ値数: {np.sum(~true_inliers)}")

# 通常の8点アルゴリズム（外れ値に弱い）
F_8point = eight_point_algorithm(pts1_noisy, pts2_noisy, normalize=True)
dist_8point = compute_epipolar_distances(F_8point, pts1_noisy, pts2_noisy)

# RANSAC
F_ransac, inlier_mask = ransac_fundamental_matrix(pts1_noisy, pts2_noisy, 
                                                   threshold=3.0, max_iterations=1000)
dist_ransac = compute_epipolar_distances(F_ransac, pts1_noisy, pts2_noisy)

print(f"\n=== 結果比較 ===")
print(f"8点アルゴリズム:")
print(f"  平均エピポーラ距離: {np.mean(dist_8point):.2f} pixels")
print(f"  中央値: {np.median(dist_8point):.2f} pixels")

print(f"\nRANSAC:")
print(f"  検出インライア数: {np.sum(inlier_mask)}")
print(f"  平均エピポーラ距離（インライアのみ）: {np.mean(dist_ransac[inlier_mask]):.2f} pixels")
print(f"  インライア検出精度: {np.sum(inlier_mask & true_inliers) / np.sum(true_inliers) * 100:.1f}%")

In [ ]:
def visualize_ransac_result(pts1, pts2, F, inlier_mask, true_inliers):
    """RANSACの結果を可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    img_size = (640, 480)
    
    # 1. 全対応点
    ax = axes[0]
    ax.set_xlim(0, img_size[0])
    ax.set_ylim(img_size[1], 0)
    ax.set_aspect('equal')
    ax.add_patch(plt.Rectangle((0, 0), img_size[0], img_size[1], 
                                 fill=True, facecolor='#f0f0f0', edgecolor='black'))
    
    # 真のインライア/アウトライア
    ax.scatter(pts2[true_inliers, 0], pts2[true_inliers, 1], 
               c='blue', s=30, alpha=0.5, label='True Inliers')
    ax.scatter(pts2[~true_inliers, 0], pts2[~true_inliers, 1], 
               c='red', s=30, alpha=0.5, label='True Outliers')
    ax.set_title('Ground Truth', fontsize=12)
    ax.legend()
    
    # 2. RANSAC検出結果
    ax = axes[1]
    ax.set_xlim(0, img_size[0])
    ax.set_ylim(img_size[1], 0)
    ax.set_aspect('equal')
    ax.add_patch(plt.Rectangle((0, 0), img_size[0], img_size[1], 
                                 fill=True, facecolor='#f0f0f0', edgecolor='black'))
    
    ax.scatter(pts2[inlier_mask, 0], pts2[inlier_mask, 1], 
               c='green', s=30, alpha=0.5, label='Detected Inliers')
    ax.scatter(pts2[~inlier_mask, 0], pts2[~inlier_mask, 1], 
               c='orange', s=30, alpha=0.5, label='Detected Outliers')
    ax.set_title('RANSAC Detection', fontsize=12)
    ax.legend()
    
    # 3. エピポーラ距離の分布
    ax = axes[2]
    distances = compute_epipolar_distances(F, pts1, pts2)
    
    ax.hist(distances[true_inliers], bins=30, alpha=0.5, label='True Inliers', color='blue')
    ax.hist(distances[~true_inliers], bins=30, alpha=0.5, label='True Outliers', color='red')
    ax.axvline(x=3.0, color='black', linestyle='--', label='Threshold (3px)')
    ax.set_xlabel('Epipolar Distance (pixels)')
    ax.set_ylabel('Count')
    ax.set_title('Epipolar Distance Distribution', fontsize=12)
    ax.legend()
    ax.set_xlim(0, 50)
    
    plt.suptitle('RANSAC Robust Estimation Results', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()

visualize_ransac_result(pts1_noisy, pts2_noisy, F_ransac, inlier_mask, true_inliers)

---

## 7. 本質行列の分解（R, tの復元）

### 7.1 理論

本質行列 $\mathbf{E}$ から回転 $\mathbf{R}$ と並進 $\mathbf{t}$ を復元できます。

#### SVD分解

$$\mathbf{E} = \mathbf{U} \text{diag}(\sigma, \sigma, 0) \mathbf{V}^\top$$

#### 4つの解候補

$$\mathbf{W} = \begin{pmatrix} 0 & -1 & 0 \\ 1 & 0 & 0 \\ 0 & 0 & 1 \end{pmatrix}$$

| 解 | 回転 | 並進 |
|---|------|------|
| 1 | $\mathbf{R} = \mathbf{U} \mathbf{W} \mathbf{V}^\top$ | $\mathbf{t} = \mathbf{u}_3$ |
| 2 | $\mathbf{R} = \mathbf{U} \mathbf{W} \mathbf{V}^\top$ | $\mathbf{t} = -\mathbf{u}_3$ |
| 3 | $\mathbf{R} = \mathbf{U} \mathbf{W}^\top \mathbf{V}^\top$ | $\mathbf{t} = \mathbf{u}_3$ |
| 4 | $\mathbf{R} = \mathbf{U} \mathbf{W}^\top \mathbf{V}^\top$ | $\mathbf{t} = -\mathbf{u}_3$ |

ここで $\mathbf{u}_3$ は $\mathbf{U}$ の3列目。

### 7.2 正しい解の選択

4つの解のうち、**両方のカメラの前方に3D点が復元される**解が正しい解です。

チェック方法：
1. 対応点から3D点を三角測量で復元
2. 復元された点のZ座標（深度）が両カメラで正であることを確認

In [ ]:
def decompose_essential_matrix(E: np.ndarray) -> List[Tuple[np.ndarray, np.ndarray]]:
    """本質行列を分解して4つのR, t候補を返す
    
    Returns:
        List of (R, t) tuples, 4 solutions
    """
    U, S, Vt = np.linalg.svd(E)
    
    # detが正になるように調整
    if np.linalg.det(U) < 0:
        U = -U
    if np.linalg.det(Vt) < 0:
        Vt = -Vt
    
    # W行列
    W = np.array([
        [0, -1, 0],
        [1, 0, 0],
        [0, 0, 1]
    ])
    
    # 4つの解候補
    R1 = U @ W @ Vt
    R2 = U @ W.T @ Vt
    t = U[:, 2]  # Uの3列目
    
    # det(R) = 1 を保証
    if np.linalg.det(R1) < 0:
        R1 = -R1
    if np.linalg.det(R2) < 0:
        R2 = -R2
    
    solutions = [
        (R1, t),
        (R1, -t),
        (R2, t),
        (R2, -t)
    ]
    
    return solutions

def triangulate_point(K: np.ndarray, R1: np.ndarray, t1: np.ndarray,
                      R2: np.ndarray, t2: np.ndarray,
                      x1: np.ndarray, x2: np.ndarray) -> np.ndarray:
    """DLT法による三角測量
    
    Args:
        K: カメラ内部パラメータ
        R1, t1: カメラ1の姿勢
        R2, t2: カメラ2の姿勢
        x1, x2: 対応する画像点
    
    Returns:
        X: 3D点の座標
    """
    # 投影行列
    P1 = K @ np.hstack([R1, t1.reshape(-1, 1)])
    P2 = K @ np.hstack([R2, t2.reshape(-1, 1)])
    
    # DLT
    A = np.array([
        x1[0] * P1[2] - P1[0],
        x1[1] * P1[2] - P1[1],
        x2[0] * P2[2] - P2[0],
        x2[1] * P2[2] - P2[1]
    ])
    
    _, _, Vt = np.linalg.svd(A)
    X_h = Vt[-1]
    X = X_h[:3] / X_h[3]
    
    return X

def check_cheirality(K: np.ndarray, R: np.ndarray, t: np.ndarray,
                     pts1: np.ndarray, pts2: np.ndarray) -> int:
    """チェイラリティ条件（カメラ前方）を満たす点の数を数える"""
    # カメラ1は原点
    R1, t1 = np.eye(3), np.zeros(3)
    R2, t2 = R, t
    
    n_valid = 0
    for i in range(min(len(pts1), 50)):  # 50点でチェック
        X = triangulate_point(K, R1, t1, R2, t2, pts1[i], pts2[i])
        
        # カメラ1座標でのZ（深度）
        z1 = X[2]
        
        # カメラ2座標での位置
        X_cam2 = R @ X + t
        z2 = X_cam2[2]
        
        if z1 > 0 and z2 > 0:
            n_valid += 1
    
    return n_valid

def recover_pose_from_essential(E: np.ndarray, K: np.ndarray,
                                 pts1: np.ndarray, pts2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """本質行列からカメラ姿勢を復元（正しい解を選択）"""
    solutions = decompose_essential_matrix(E)
    
    best_R, best_t = None, None
    best_count = 0
    
    print("=== 4つの解候補の評価 ===")
    for i, (R, t) in enumerate(solutions):
        count = check_cheirality(K, R, t, pts1, pts2)
        print(f"解{i+1}: 有効点数 = {count}")
        
        if count > best_count:
            best_count = count
            best_R, best_t = R, t
    
    print(f"\n選択された解: 有効点数 = {best_count}")
    
    return best_R, best_t

print("本質行列分解の実装完了")

In [ ]:
# 本質行列の計算（Fから）
E_estimated = K.T @ F_ransac @ K

# 特異値の正規化（σ, σ, 0の形式に）
U, S, Vt = np.linalg.svd(E_estimated)
S_corrected = np.array([(S[0] + S[1]) / 2, (S[0] + S[1]) / 2, 0])
E_corrected = U @ np.diag(S_corrected) @ Vt

print("推定された本質行列 E:")
print(E_corrected)

# 姿勢の復元
R_recovered, t_recovered = recover_pose_from_essential(E_corrected, K, pts1_noisy[inlier_mask], pts2_noisy[inlier_mask])

print("\n=== 復元された姿勢 ===")
print("回転行列 R:")
print(R_recovered)
print(f"\n並進ベクトル t: {t_recovered}")
print(f"並進方向（正規化）: {t_recovered / np.linalg.norm(t_recovered)}")

# 真の値との比較
print("\n=== 真の値との比較 ===")
print("真の回転行列 R:")
print(R)
print(f"\n真の並進方向（正規化）: {t / np.linalg.norm(t)}")

# 回転の誤差（角度）
R_error = R_recovered @ R.T
angle_error = np.arccos(np.clip((np.trace(R_error) - 1) / 2, -1, 1))
print(f"\n回転誤差: {np.degrees(angle_error):.2f}度")

# 並進方向の誤差（角度）
t_true_norm = t / np.linalg.norm(t)
t_recovered_norm = t_recovered / np.linalg.norm(t_recovered)
# 符号の不定性を考慮
cos_angle = np.abs(np.dot(t_true_norm, t_recovered_norm))
translation_angle_error = np.arccos(np.clip(cos_angle, -1, 1))
print(f"並進方向誤差: {np.degrees(translation_angle_error):.2f}度")

In [ ]:
def visualize_pose_recovery():
    """復元されたカメラ姿勢の可視化"""
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # カメラ1（原点）
    C1 = np.array([0, 0, 0])
    
    # 真のカメラ2位置（並進ベクトルはカメラ座標での相対位置）
    # C2 = -R^T @ t
    C2_true = -R.T @ t
    
    # 復元されたカメラ2位置
    C2_recovered = -R_recovered.T @ t_recovered
    # スケール調整（真の並進距離に合わせる）
    scale = np.linalg.norm(C2_true) / np.linalg.norm(C2_recovered)
    C2_recovered_scaled = C2_recovered * scale
    
    # カメラの描画
    def draw_camera(ax, C, R, color, label, scale=0.3):
        # カメラの向き（Z軸方向）
        z_dir = R.T @ np.array([0, 0, 1])
        x_dir = R.T @ np.array([1, 0, 0])
        y_dir = R.T @ np.array([0, 1, 0])
        
        ax.scatter(*C, color=color, s=200, label=label)
        ax.quiver(*C, *z_dir * scale, color=color, arrow_length_ratio=0.2, linewidth=2)
        ax.quiver(*C, *x_dir * scale * 0.5, color=color, alpha=0.5, arrow_length_ratio=0.2)
        ax.quiver(*C, *y_dir * scale * 0.5, color=color, alpha=0.5, arrow_length_ratio=0.2)
    
    # カメラ1
    draw_camera(ax, C1, np.eye(3), 'blue', 'Camera 1')
    
    # 真のカメラ2
    draw_camera(ax, C2_true, R, 'green', 'Camera 2 (True)')
    
    # 復元されたカメラ2
    draw_camera(ax, C2_recovered_scaled, R_recovered, 'red', 'Camera 2 (Recovered)')
    
    # 3D点（一部）
    if len(X_3d) > 0:
        ax.scatter(X_3d[:, 0], X_3d[:, 1], X_3d[:, 2], 
                   c='gray', s=20, alpha=0.3, label='3D Points')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Camera Pose Recovery from Essential Matrix', fontsize=14)
    ax.legend()
    
    # 等軸スケール
    max_range = np.array([X_3d[:, 0].max() - X_3d[:, 0].min(),
                          X_3d[:, 1].max() - X_3d[:, 1].min(),
                          X_3d[:, 2].max() - X_3d[:, 2].min()]).max() / 2.0
    mid_x = (X_3d[:, 0].max() + X_3d[:, 0].min()) * 0.5
    mid_y = (X_3d[:, 1].max() + X_3d[:, 1].min()) * 0.5
    mid_z = (X_3d[:, 2].max() + X_3d[:, 2].min()) * 0.5
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)
    
    ax.view_init(elev=20, azim=-60)
    
    plt.tight_layout()
    plt.show()

visualize_pose_recovery()

---

## 8. 実践的な注意点

### 8.1 退化条件（Degenerate Cases）

エピポーラ幾何が決定できない状況：

| 条件 | 説明 | 対処法 |
|------|------|--------|
| **純回転** | カメラが回転のみで移動なし（$\mathbf{t} = \mathbf{0}$） | ホモグラフィを使用 |
| **平面シーン** | 全点が同一平面上 | ホモグラフィを使用 |
| **共線点** | 全点が一直線上 | 避ける |

### 8.2 数値安定性

1. **点の正規化**: 8点アルゴリズム前に必須
2. **RANSAC**: 外れ値に対するロバスト性
3. **非線形最適化**: 初期値を線形解で与え、再投影誤差を最小化

### 8.3 スケールの不定性

単眼カメラからは**絶対スケール**を復元できません。

解決方法：
- 既知の距離（オブジェクトサイズ、ベースライン長）
- IMU との融合
- ステレオカメラの使用

In [ ]:
def demonstrate_degenerate_case():
    """退化条件のデモ: 純回転ケース"""
    print("=== 退化条件: 純回転（t = 0）===")
    
    # 純回転（並進なし）
    theta = np.radians(15)
    R_pure = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    t_zero = np.array([0, 0, 0])
    
    # 本質行列
    E_pure = compute_essential_matrix(R_pure, t_zero)
    
    print("本質行列 E (純回転):")
    print(E_pure)
    print(f"\n||E|| = {np.linalg.norm(E_pure):.6f}")
    print("→ E ≈ 0 となり、エピポーラ幾何が退化します")
    
    print("\n解決策: ホモグラフィ行列 H を使用")
    print("平面上の点に対して: x2 = H @ x1")

demonstrate_degenerate_case()

---

## 9. まとめと次のステップ

### 学んだこと

1. **エピポーラ幾何の基本**: エピポール、エピポーラ線、エピポーラ面
2. **基礎行列 F**: 画像座標間のエピポーラ拘束 $\mathbf{x}_2^\top \mathbf{F} \mathbf{x}_1 = 0$
3. **本質行列 E**: 正規化座標間の拘束、$\mathbf{E} = [\mathbf{t}]_\times \mathbf{R}$
4. **8点アルゴリズム**: 対応点からFを推定
5. **RANSAC**: 外れ値に対するロバスト推定
6. **Eの分解**: カメラの相対姿勢（R, t）の復元

### 重要な数式

| 概念 | 数式 |
|------|------|
| エピポーラ拘束（F） | $\mathbf{x}_2^\top \mathbf{F} \mathbf{x}_1 = 0$ |
| エピポーラ拘束（E） | $\hat{\mathbf{x}}_2^\top \mathbf{E} \hat{\mathbf{x}}_1 = 0$ |
| F と E の関係 | $\mathbf{F} = \mathbf{K}_2^{-\top} \mathbf{E} \mathbf{K}_1^{-1}$ |
| E の構造 | $\mathbf{E} = [\mathbf{t}]_\times \mathbf{R}$ |
| エピポーラ線 | $\mathbf{l}_2 = \mathbf{F} \mathbf{x}_1$ |

### 次のノートブック

**56. ステレオ視と視差**では：
- ステレオ画像ペアの整列（Rectification）
- 視差（Disparity）マップの計算
- ブロックマッチングアルゴリズム
- 深度マップの生成

---

## 10. 自己評価クイズ

以下の質問に答えて理解度を確認しましょう：

1. **エピポール** $\mathbf{e}_1$ は何を表しますか？

2. エピポーラ拘束 $\mathbf{x}_2^\top \mathbf{F} \mathbf{x}_1 = 0$ の幾何学的な意味は？

3. 基礎行列 $\mathbf{F}$ のランクが2である理由は？

4. 本質行列 $\mathbf{E}$ と基礎行列 $\mathbf{F}$ の違いは何ですか？

5. 8点アルゴリズムで「正規化」が重要な理由は？

6. RANSACが必要な理由は何ですか？

7. 本質行列の分解で4つの解候補が出る理由と、正しい解を選ぶ方法は？

8. 純回転（$\mathbf{t} = \mathbf{0}$）の場合、なぜエピポーラ幾何が退化するのですか？

In [ ]:
# クイズの解答（隠し）
def show_quiz_answers():
    answers = """
    === 自己評価クイズ解答 ===
    
    1. エピポール e₁ は、カメラ2の光学中心をカメラ1の画像面に投影した点です。
       ベースラインと画像面の交点とも言えます。
    
    2. 画像1の点x₁と対応する点x₂は、画像2上のエピポーラ線l₂ = Fx₁上に
       必ず存在するという拘束条件を表します。
    
    3. rank(F) = 2 である理由：
       - Fはエピポールを零空間に持つ（Fe₁ = 0, F^Te₂ = 0）
       - これにより1つの特異値が0になり、ランクが2に制限される
    
    4. 違い：
       - F: ピクセル座標で定義、カメラ内部パラメータに依存
       - E: 正規化座標で定義、幾何学的情報（R, t）のみを含む
       - 関係: E = K₂ᵀFK₁
    
    5. 正規化の重要性：
       - ピクセル座標の値の範囲が大きく、行列Aの条件数が悪化する
       - 正規化により数値的に安定した解が得られる
    
    6. RANSAC が必要な理由：
       - 実データには外れ値（誤対応）が含まれる
       - 8点アルゴリズムは最小二乗法ベースで外れ値に敏感
       - RANSACはロバストに正しい解を見つけられる
    
    7. 4つの解候補：
       - SVD分解の符号の不定性から4通りの(R, t)組み合わせが生じる
       - 正しい解の選択：三角測量で3D点を復元し、両カメラの前方（Z > 0）
         にある点が最も多い解を選ぶ（チェイラリティ条件）
    
    8. 純回転での退化：
       - t = 0 のとき、E = [t]×R = 0 となる
       - ベースラインが存在しないため、エピポーラ幾何が定義できない
       - 全ての点が同じ光線上に投影され、深度情報が失われる
    """
    print(answers)

# 解答を見るには以下のコメントを外して実行
# show_quiz_answers()

---

## ナビゲーション

- **前のノートブック**: [54. カメラキャリブレーション](54_camera_calibration_v1.ipynb)
- **次のノートブック**: [56. ステレオ視と視差](56_stereo_vision_disparity_v1.ipynb)
- **カリキュラム**: [CURRICULUM_UNIT_0.3.md](CURRICULUM_UNIT_0.3.md)